In [1]:
import pandas as pd
import numpy as np
import pickle
import os, sys
import re
import glob

#
import pathmagic
import initialise as init

In [27]:
pd.set_option('display.max_columns', 100)

### Accident data

In [36]:
# load data
all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-accident-2016.csv")

acc_files = [pd.read_csv(file) for file in all_files] 

df_acc = pd.concat(acc_files)

C:\Users\farqu\AppData\Local\Temp/ipykernel_4784/1504224073.py:4: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  acc_files = [pd.read_csv(file) for file in all_files]


In [37]:
# check if missings (and if all in same year - file)
df_acc[df_acc.isna().any(axis =1)].groupby('accident_year')['accident_index'].count()

accident_year
2016    44
Name: accident_index, dtype: int64

In [38]:
df_acc.head(20)

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2016010000005,2016,010000005,519310.0,188730.0,-0.279323,51.584754,1,3,2,1,01/11/2016,3,02:30,28,E09000005,E09000005,3,4006,6,30.0,0,-1,-1,-1,0,0,5,1,1,0,0,1,1,2,E01000543
1,2016010000006,2016,010000006,551920.0,174560.0,0.184928,51.449595,1,3,1,1,01/11/2016,3,00:37,18,E09000004,E09000004,3,207,6,30.0,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000375
2,2016010000008,2016,010000008,505930.0,183850.0,-0.473837,51.543563,1,3,1,1,01/11/2016,3,01:25,26,E09000017,E09000017,3,4020,1,30.0,1,4,3,4020,0,0,4,1,1,0,0,1,1,2,E01033725
3,2016010000016,2016,010000016,527770.0,168930.0,-0.164442,51.404958,1,3,1,1,01/11/2016,3,09:15,22,E09000024,E09000024,3,217,6,30.0,3,2,3,217,0,0,1,1,1,0,0,1,1,2,E01003379
4,2016010000018,2016,010000018,510740.0,177230.0,-0.406580,51.483139,1,3,2,1,01/11/2016,3,07:53,25,E09000018,E09000018,3,312,3,40.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01002583
5,2016010000019,2016,010000019,530840.0,167100.0,-0.121004,51.387813,1,3,2,1,01/11/2016,3,09:29,20,E09000008,E09000008,3,23,6,30.0,3,4,6,0,0,8,1,1,1,0,0,1,1,2,E01001180
6,2016010000020,2016,010000020,527610.0,181430.0,-0.162234,51.517333,1,3,1,1,01/11/2016,3,08:53,1,E09000033,E09000033,5,0,6,30.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01004658
7,2016010000021,2016,010000021,523830.0,189460.0,-0.213859,51.590341,1,3,2,1,01/11/2016,3,10:05,30,E09000003,E09000003,3,504,6,30.0,3,4,6,0,0,0,1,1,1,0,0,1,1,2,E01000243
8,2016010000023,2016,010000023,524640.0,180620.0,-0.205302,51.510716,1,3,1,1,01/11/2016,3,09:51,12,E09000020,E09000020,4,450,6,30.0,3,4,6,0,0,0,1,1,1,0,0,1,1,2,E01002875
9,2016010000024,2016,010000024,540670.0,174700.0,0.023193,51.453758,1,3,2,1,01/11/2016,3,09:45,6,E09000011,E09000011,3,20,6,30.0,3,4,6,0,0,0,1,1,1,0,0,1,1,2,E01001658


* area key col: lsoa_of_accident_location

### Casualty data

In [14]:
# load data
all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-casualty-2016.csv")

cas_files = [pd.read_csv(file) for file in all_files] 

df_cas = pd.concat(cas_files)

In [15]:
df_cas.head()

,accident_index,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile
0,2016010000005,2016,010000005,2,1,1,1,23,5,3,0,0,0,0,0,2,1,4
1,2016010000006,2016,010000006,1,1,1,2,36,7,3,0,0,0,0,0,9,1,10
2,2016010000008,2016,010000008,1,1,1,1,24,5,3,0,0,0,0,0,9,1,8
3,2016010000016,2016,010000016,1,1,2,2,59,9,3,0,0,0,3,0,11,1,4
4,2016010000018,2016,010000018,2,1,1,1,28,6,3,0,0,0,0,0,1,1,6


* x rows per accidentr_index for each casualty (reference)
* merge on casualty file by accident_index

In [17]:
# load data
all_files = glob.glob(init.raw_path + "/road_safety_data/dft-road-casualty-statistics-vehicle-2016.csv")

veh_files = [pd.read_csv(file) for file in all_files] 

df_veh = pd.concat(veh_files)



In [19]:
df_veh.head()

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,...,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type
0,2016010000005,2016,010000005,1,8,0,6,3,7,0,...,1,1,51,8,-1,-1,-1,-1,3,1
1,2016010000005,2016,010000005,2,2,0,18,7,3,0,...,6,1,23,5,124,1,4,-1,4,1
2,2016010000006,2016,010000006,1,9,0,18,5,1,0,...,6,2,36,7,1461,2,1,-1,10,1
3,2016010000008,2016,010000008,1,9,0,18,1,8,0,...,6,1,24,5,1390,1,5,-1,8,1
4,2016010000016,2016,010000016,1,11,0,18,5,1,0,...,1,1,55,8,6692,2,7,-1,9,1


* x rows per accidentr_index for each vehicle (reference)
* merge on veh data by accident index

## combine data

In [41]:
master_df = pd.merge(left = df_acc.head(20), on = 'accident_index', right = df_veh.head(20), how = 'left')
master_df = pd.merge(left = master_df, on = 'accident_index', right = df_cas.head(20), how = 'left')

In [42]:
master_df

,accident_index,accident_year_x,accident_reference_x,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,date,day_of_week,time,local_authority_district,local_authority_ons_district,local_authority_highway,first_road_class,first_road_number,road_type,speed_limit,junction_detail,junction_control,second_road_class,second_road_number,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location,accident_year_y,accident_reference_y,vehicle_reference_x,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,junction_location,skidding_and_overturning,hit_object_in_carriageway,vehicle_leaving_carriageway,hit_object_off_carriageway,first_point_of_impact,vehicle_left_hand_drive,journey_purpose_of_driver,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,accident_year,accident_reference,vehicle_reference_y,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile
0,2016010000005,2016,010000005,519310.0,188730.0,-0.279323,51.584754,1,3,2,1,01/11/2016,3,02:30,28,E09000005,E09000005,3,4006,6,30.0,0,-1,-1,-1,0,0,5,1,1,0,0,1,1,2,E01000543,2016.0,010000005,1.0,8.0,0.0,6.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,51.0,8.0,-1.0,-1.0,-1.0,-1.0,3.0,1.0,2016.0,010000005,2.0,1.0,1.0,1.0,23.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0
1,2016010000005,2016,010000005,519310.0,188730.0,-0.279323,51.584754,1,3,2,1,01/11/2016,3,02:30,28,E09000005,E09000005,3,4006,6,30.0,0,-1,-1,-1,0,0,5,1,1,0,0,1,1,2,E01000543,2016.0,010000005,2.0,2.0,0.0,18.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,23.0,5.0,124.0,1.0,4.0,-1.0,4.0,1.0,2016.0,010000005,2.0,1.0,1.0,1.0,23.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0
2,2016010000006,2016,010000006,551920.0,174560.0,0.184928,51.449595,1,3,1,1,01/11/2016,3,00:37,18,E09000004,E09000004,3,207,6,30.0,9,4,6,0,0,0,4,1,1,0,0,1,1,2,E01000375,2016.0,010000006,1.0,9.0,0.0,18.0,5.0,1.0,0.0,2.0,2.0,10.0,1.0,0.0,4.0,1.0,6.0,2.0,36.0,7.0,1461.0,2.0,1.0,-1.0,10.0,1.0,2016.0,010000006,1.0,1.0,1.0,2.0,36.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,10.0
3,2016010000008,2016,010000008,505930.0,183850.0,-0.473837,51.543563,1,3,1,1,01/11/2016,3,01:25,26,E09000017,E09000017,3,4020,1,30.0,1,4,3,4020,0,0,4,1,1,0,0,1,1,2,E01033725,2016.0,010000008,1.0,9.0,0.0,18.0,1.0,8.0,0.0,4.0,5.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,24.0,5.0,1390.0,1.0,5.0,-1.0,8.0,1.0,2016.0,010000008,1.0,1.0,1.0,1.0,24.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,8.0
4,2016010000016,2016,010000016,527770.0,168930.0,-0.164442,51.404958,1,3,1,1,01/11/2016,3,09:15,22,E09000024,E09000024,3,217,6,30.0,3,2,3,217,0,0,1,1,1,0,0,1,1,2,E01003379,2016.0,010000016,1.0,11.0,0.0,18.0,5.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,55.0,8.0,6692.0,2.0,7.0,-1.0,9.0,1.0,2016.0,010000016,1.0,1.0,2.0,2.0,59.0,9.0,3.0,0.0,0.0,0.0,3.0,0.0,11.0,1.0,4.0
5,2016010000018,2016,010000018,510740.0,177230.0,-0.406580,51.483139,1,3,2,1,01/11/2016,3,07:53,25,E09000018,E09000018,3,312,3,40.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01002583,2016.0,010000018,1.0,9.0,0.0,12.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0,1.0,20.0,4.0,1596.0,1.0,11.0,-1.0,2.0,1.0,2016.0,010000018,2.0,1.0,1.0,1.0,28.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6.0
6,2016010000018,2016,010000018,510740.0,177230.0,-0.406580,51.483139,1,3,2,1,01/11/2016,3,07:53,25,E09000018,E09000018,3,312,3,40.0,0,-1,-1,-1,0,0,1,1,1,0,0,1,1,2,E01002583,2016.0,010000018,2.0,1.0,0.0,18.0,

* multiple rows per accident if multiple vehicles or casualties

Gives us all accidents from 2016 to 2021 with:
* accident info (when, where, why, situation)
* collision victim(s) information if any: who, age, civilian type etc
* vehicle(s) and driver(s) information if any: vehicle situation, type, driver info